### **Sentiment Analysis for Reddit and News data and merging with Stock prices data**

#### Imporing Required Libraries

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

### **Note:** Before running the cell below, ensure that all files from the `Sentiment_Analysis` folder are uploaded to the session storage.

In [ ]:
tsla_data = pd.read_csv('/content/tesla_stocks_18_to_24.csv', on_bad_lines='skip')
aapl_data = pd.read_csv('/content/aapl_stocks_18_to_24.csv', on_bad_lines='skip')

In [ ]:
tsla_data.set_index('Date', inplace=True)
aapl_data.set_index('Date', inplace=True)

In [ ]:
tsla_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,20.799999,21.474001,20.733334,21.368668,21.368668,65283000
2018-01-03,21.400000,21.683332,21.036667,21.150000,21.150000,67822500
2018-01-04,20.858000,21.236668,20.378668,20.974667,20.974667,149194500
2018-01-05,21.108000,21.149332,20.799999,21.105333,21.105333,68868000
2018-01-08,21.066668,22.468000,21.033333,22.427334,22.427334,147891000
...,...,...,...,...,...,...
2024-11-22,341.089996,361.529999,337.700012,352.559998,352.559998,89140700
2024-11-25,360.140015,361.929993,338.200012,338.589996,338.589996,95890900
2024-11-26,341.000000,346.959991,335.660004,338.230011,338.230011,62295900


In [ ]:
aapl_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,42.540001,43.075001,42.314999,43.064999,40.524345,102223600
2018-01-03,43.132500,43.637501,42.990002,43.057499,40.517288,118071600
2018-01-04,43.134998,43.367500,43.020000,43.257500,40.705486,89738400
2018-01-05,43.360001,43.842499,43.262501,43.750000,41.168930,94640000
2018-01-08,43.587502,43.902500,43.482498,43.587502,41.016026,82271200
...,...,...,...,...,...,...
2024-11-22,228.059998,230.720001,228.059998,229.869995,229.869995,38168300
2024-11-25,231.460007,233.250000,229.740005,232.869995,232.869995,90152800
2024-11-26,233.330002,235.570007,233.330002,235.059998,235.059998,45986200


In [ ]:
tsla_reddit_data = pd.read_csv('/content/tesla_reddit_data_18_to_24.csv', on_bad_lines='skip')
tsla_news_data = pd.read_csv('/content/tesla_news_data_18_to_24.csv', on_bad_lines='skip')

In [ ]:
aapl_reddit_data = pd.read_csv('/content/aapl_reddit_data_18_to_24.csv', on_bad_lines='skip')
aapl_news_data = pd.read_csv('/content/aapl_news_data_18_to_24.csv', on_bad_lines='skip')

In [ ]:
tsla_reddit_data

,title,selftext,created_utc
0,Bi-weekly TSLA Investor Thread,This will post every other Monday (EST) at 6AM...,2018-01-01 11:14:30
1,Long $TSLA,NaN,2018-01-01 18:44:21
2,Get ready to short $TSLA - they finally added ...,NaN,2018-01-01 20:41:06
3,TSLA,[removed],2018-01-01 21:04:17
4,Quick 1/1/18 drive-by looks like Marina Del Re...,[deleted],2018-01-01 23:52:28
...,...,...,...
83736,Those who think removing the EV tax credit wil...,"1. Trump removes $7,500 EV tax credits and imp...",2024-11-15 06:49:43
83737,Coping with Loss in the Time of 🥭 (TSLA and LUNR),https://preview.redd.it/tk91sh5tpy0e1.jpg?widt...,2024-11-15 00:49:00
83738,"South Korea's ""Value Up"" program, and why Kore...","Korean stocks have long been undervalued, refe...",2024-11-14 18:51:15
83739,Comment,Open the fucking markets my Tesla calls are re...,2024-11-30 23:58:49


In [ ]:
tsla_news_data

,url,seendate,title,domain
0,https://kldaily.com/tesla-inc-tsla-eps-estimat...,20180129T074500Z,"Tesla , Inc . ( TSLA ) EPS Estimated At $ - 3 ...",kldaily.com
1,https://friscofastball.com/analysts-see-3-75-e...,20180129T011500Z,"Analysts See $ - 3 . 75 EPS for Tesla , Inc . ...",friscofastball.com
2,http://www.benchmarkmonitor.com/cnbc-a-key-eng...,20180130T111500Z,CNBC ; A key Engineer at Tesla Motors Inc ( ...,benchmarkmonitor.com
3,https://santimes.com/cousins-properties-cuz-an...,20180115T161500Z,Cousins Properties ( CUZ ) Analysts See $0 . 1...,santimes.com
4,https://ledgergazette.com/2018/01/19/insider-s...,20180119T063000Z,Tesla Inc ( NASDAQ : TSLA ) VP John Douglas Fi...,ledgergazette.com
...,...,...,...,...
21245,http://www.itbear.com.cn/html/2024-11/609574.html,20241125T140000Z,特斯拉中国罕见优惠 ： Model Y最高减10000元 ！- 智能汽车 - ITBear科技资讯,itbear.com.cn
21246,https://www.ideastream.org/2024-11-22/tesla-wo...,20241125T041500Z,Tesla won the plug war . Enter the age of the ...,ideastream.org
21247,https://www.cnet.com/paid-content/news/complet...,20241125T163000Z,Complete Your Home Energy Ecosystem With Tesla...,cnet.com
21248,https://finance.sina.com.cn/jjxw/2024-11-30/do...,20241130T093000Z,极越汽车被指抄袭特斯拉 ？ CEO夏一平 ： 只是自动驾驶理念相同,finance.sina.com.cn


In [ ]:
aapl_reddit_data

,title,selftext,created_utc
0,Blowing versus sucking,AAPL just entered a contract to purchase 51 of...,2018-01-01 01:04:25
1,The 2018 /r/Robinhood Stock Picking Game,# tl;dr\n\n - Stock picking game will last all...,2018-01-01 16:37:41
2,Hesitant to invest in $AAPL,Looking at AAPLs fundamentals and the pile of ...,2018-01-01 20:21:33
3,Stock mix help required (ETF),I’ve decided I’m most likely interested in jus...,2018-01-01 21:54:14
4,ETF advice please!,I’ve decided I’m most likely interested in jus...,2018-01-01 22:12:01
...,...,...,...
44615,2 month rollercoaster: 2k>55k > -25k > 125k,After not trading for a 2 years to save up for...,2024-11-21 16:15:48
44616,$ACHR The Bull Run Hasn't Started Yet,"**TLDR:** Current fair value is +$10imo, Arch...",2024-11-20 15:02:27
44617,Trump taps big tech critic Carr to lead US com...,"How do you think MSFT, META, GOOGL, and AAPL w...",2024-11-18 15:26:03
44618,SoundHound AI Stock Tumbles as Margins Drop,Was looking more into earnings and what could ...,2024-11-13 21:08:42


In [ ]:
aapl_news_data

,url,seendate,title,domain
0,http://weeklyregister.com/british-columbia-inv...,20180121T120000Z,British Columbia Investment Management Corp Cu...,weeklyregister.com
1,https://bzweekly.com/2018/01/13/can-apple-inc-...,20180114T013000Z,Can Apple Inc . ( AAPL ) s Tomorrow be Differe...,bzweekly.com
2,https://kldaily.com/could-apple-inc-aapl-chang...,20180117T211500Z,Could Apple Inc . ( AAPL ) Change Direction Af...,kldaily.com
3,https://friscofastball.com/eaton-vance-managem...,20180120T131500Z,Eaton Vance Management Has Trimmed Position in...,friscofastball.com
4,http://finnewsdaily.com/as-apple-inc-aapl-stoc...,20180120T133000Z,"As Apple INC ( AAPL ) Stock Price Rose , Share...",finnewsdaily.com
...,...,...,...,...
20995,https://telegrafi.com/en/apple-zgjeron-kampanj...,20241125T060000Z,Apple expands There more in an iPhone campai...,telegrafi.com
20996,http://kr.xinhuanet.com/20241127/f8f050da74194...,20241127T071500Z,"팀 쿡 애플 CEO 中 , 세계에서 가장 중요한 공급사슬 - Xinhua",kr.xinhuanet.com
20997,https://www.annapurnapost.com/story/469101/,20241126T164500Z,मुस्ताङको कृषि उत्पादनमा पर्यटकको क्रेज,annapurnapost.com
20998,https://www.macworld.com/article/2535956/the-i...,20241127T201500Z,The next iPhone SE will be a study in sacrifice,macworld.com


In [ ]:
def get_reddit_data_with_sentiment_analysis(data):
    # Check if a GPU is available and set the device
    device = 0 if torch.cuda.is_available() else -1  # 0 for GPU, -1 for CPU

    # Load the FinBERT model and tokenizer
    model_name = "yiyanghkust/finbert-tone"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Create a sentiment analysis pipeline with the specified device
    sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)

    # Fill NaN values in selftext with an empty string
    data['selftext'] = data['selftext'].fillna("")

    # Combine title and selftext for sentiment analysis
    data['text'] = data['title'] + " " + data['selftext']

    # Function to apply sentiment analysis
    def get_sentiment(text):
        # Apply sentiment analysis and return the label and score
        result = sentiment_analyzer(text[:512])[0]  # Limiting to 512 characters for BERT model
        return result['label'], result['score']

    # Apply sentiment analysis on the text column
    data[['sentiment', 'sentiment_score']] = data['text'].apply(lambda x: pd.Series(get_sentiment(x)))

    return data

In [ ]:
def get_news_data_with_sentiment_analysis(data):
    device = 0 if torch.cuda.is_available() else -1  # 0 for GPU, -1 for CPU

    # Load the FinBERT model and tokenizer
    model_name = "yiyanghkust/finbert-tone"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Create a sentiment analysis pipeline with the specified device
    sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)

    # Handle missing or invalid values in the title column
    data['title'] = data['title'].fillna("").astype(str)

    # Function to apply sentiment analysis
    def get_sentiment(text):
        # Ensure the input is a valid string
        if not isinstance(text, str) or not text.strip():
            return "neutral", 0.0  # Default sentiment for invalid input

        # Apply sentiment analysis and return the label and score
        result = sentiment_analyzer(text[:512])[0]  # Limiting to 512 characters
        return result['label'], result['score']

    # Apply sentiment analysis on the title column
    data[['sentiment', 'sentiment_score']] = data['title'].apply(lambda x: pd.Series(get_sentiment(x)))

    return data

In [ ]:
tsla_reddit_data_with_sentiment_analysis = get_reddit_data_with_sentiment_analysis(tsla_reddit_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
tsla_news_data_with_sentiment_analysis = get_news_data_with_sentiment_analysis(tsla_news_data)

In [ ]:
aapl_reddit_data_with_sentiment_analysis = get_reddit_data_with_sentiment_analysis(aapl_reddit_data)

In [ ]:
aapl_news_data_with_sentiment_analysis = get_news_data_with_sentiment_analysis(aapl_news_data)

In [ ]:
# Ensure the 'created_utc' and 'seendate' columns are in datetime format
tsla_reddit_data_with_sentiment_analysis['created_utc'] = pd.to_datetime(
    tsla_reddit_data_with_sentiment_analysis['created_utc'], errors='coerce'
)  # Use errors='coerce' to handle invalid entries
tsla_news_data_with_sentiment_analysis['seendate'] = pd.to_datetime(
    tsla_news_data_with_sentiment_analysis['seendate'], errors='coerce'
)

# Drop rows with NaT (if datetime conversion fails for any rows)
tsla_reddit_data_with_sentiment_analysis.dropna(subset=['created_utc'], inplace=True)
tsla_news_data_with_sentiment_analysis.dropna(subset=['seendate'], inplace=True)

# Set the 'created_utc' and 'seendate' columns as index
tsla_reddit_data_with_sentiment_analysis.set_index('created_utc', inplace=True)
tsla_news_data_with_sentiment_analysis.set_index('seendate', inplace=True)

# Aggregate social media data (Reddit)
reddit_data_aggregated = tsla_reddit_data_with_sentiment_analysis.groupby(pd.Grouper(freq='B')).agg({
    'sentiment': 'max',
}).rename(columns={'sentiment': 'reddit_sentiment'})

# Aggregate news data
news_data_aggregated = tsla_news_data_with_sentiment_analysis.groupby(pd.Grouper(freq='B')).agg({
    'sentiment': 'max',
}).rename(columns={'sentiment': 'news_sentiment'})

# Ensure the indices are in datetime format and normalized
tsla_data.index = pd.to_datetime(tsla_data.index).normalize()
reddit_data_aggregated.index = pd.to_datetime(reddit_data_aggregated.index).normalize()
news_data_aggregated.index = pd.to_datetime(news_data_aggregated.index).normalize()

# Ensure all indices are timezone-aware and localized to UTC
tsla_data.index = tsla_data.index.tz_localize('UTC')
reddit_data_aggregated.index = reddit_data_aggregated.index.tz_localize('UTC')

# Merge datasets
tsla_merged_data = tsla_data.merge(reddit_data_aggregated, how='left', left_index=True, right_index=True)
tsla_merged_data = tsla_merged_data.merge(news_data_aggregated, how='left', left_index=True, right_index=True)

In [ ]:
tsla_merged_data.reset_index(inplace=True)

# Convert 'Date' column to contain only the date part
tsla_merged_data['Date'] = tsla_merged_data['Date'].dt.date

In [ ]:
tsla_merged_data

,Date,Open,High,Low,Close,Adj Close,Volume,reddit_sentiment,news_sentiment
0,2018-01-02,20.799999,21.474001,20.733334,21.368668,21.368668,65283000,Neutral,Positive
1,2018-01-03,21.400000,21.683332,21.036667,21.150000,21.150000,67822500,Positive,Neutral
2,2018-01-04,20.858000,21.236668,20.378668,20.974667,20.974667,149194500,Positive,Positive
3,2018-01-05,21.108000,21.149332,20.799999,21.105333,21.105333,68868000,Neutral,Positive
4,2018-01-08,21.066668,22.468000,21.033333,22.427334,22.427334,147891000,Neutral,Positive
...,...,...,...,...,...,...,...,...,...
1735,2024-11-22,341.089996,361.529999,337.700012,352.559998,352.559998,89140700,Positive,Positive
1736,2024-11-25,360.140015,361.929993,338.200012,338.589996,338.589996,95890900,Positive,Positive
1737,2024-11-26,341.000000,346.959991,335.660004,338.230011,338.230011,62295900,Positive,Positive
1738,2024-11-27,341.799988,342.549988,326.589996,332.890015,332.890015,57896400,Neutral,Positive


In [ ]:
# Ensure the 'created_utc' and 'seendate' columns are in datetime format
aapl_reddit_data_with_sentiment_analysis['created_utc'] = pd.to_datetime(
    aapl_reddit_data_with_sentiment_analysis['created_utc'], errors='coerce'
)  # Use errors='coerce' to handle invalid entries
aapl_news_data_with_sentiment_analysis['seendate'] = pd.to_datetime(
    aapl_news_data_with_sentiment_analysis['seendate'], errors='coerce'
)

# Drop rows with NaT (if datetime conversion fails for any rows)
aapl_reddit_data_with_sentiment_analysis.dropna(subset=['created_utc'], inplace=True)
aapl_news_data_with_sentiment_analysis.dropna(subset=['seendate'], inplace=True)

# Set the 'created_utc' and 'seendate' columns as index
aapl_reddit_data_with_sentiment_analysis.set_index('created_utc', inplace=True)
aapl_news_data_with_sentiment_analysis.set_index('seendate', inplace=True)

# Aggregate social media data (Reddit)
reddit_data_aggregated = aapl_reddit_data_with_sentiment_analysis.groupby(pd.Grouper(freq='B')).agg({
    'sentiment': 'max',
}).rename(columns={'sentiment': 'reddit_sentiment'})

# Aggregate news data
news_data_aggregated = aapl_news_data_with_sentiment_analysis.groupby(pd.Grouper(freq='B')).agg({
    'sentiment': 'max',
}).rename(columns={'sentiment': 'news_sentiment'})

# Ensure the indices are in datetime format and normalized
aapl_data.index = pd.to_datetime(aapl_data.index).normalize()
reddit_data_aggregated.index = pd.to_datetime(reddit_data_aggregated.index).normalize()
news_data_aggregated.index = pd.to_datetime(news_data_aggregated.index).normalize()

# Ensure all indices are timezone-aware and localized to UTC
aapl_data.index = aapl_data.index.tz_localize('UTC')
reddit_data_aggregated.index = reddit_data_aggregated.index.tz_localize('UTC')

# Merge datasets
aapl_merged_data = aapl_data.merge(reddit_data_aggregated, how='left', left_index=True, right_index=True)
aapl_merged_data = aapl_merged_data.merge(news_data_aggregated, how='left', left_index=True, right_index=True)

In [ ]:
aapl_merged_data.reset_index(inplace=True)

In [ ]:
# Convert 'Date' column to contain only the date part
aapl_merged_data['Date'] = aapl_merged_data['Date'].dt.date

In [ ]:
aapl_merged_data

,Date,Open,High,Low,Close,Adj Close,Volume,reddit_sentiment,news_sentiment
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,40.524345,102223600,Positive,Positive
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,40.517288,118071600,Neutral,Positive
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,40.705486,89738400,Neutral,Neutral
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.168930,94640000,Positive,Positive
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.016026,82271200,Positive,Positive
...,...,...,...,...,...,...,...,...,...
1735,2024-11-22,228.059998,230.720001,228.059998,229.869995,229.869995,38168300,Positive,Positive
1736,2024-11-25,231.460007,233.250000,229.740005,232.869995,232.869995,90152800,Neutral,Positive
1737,2024-11-26,233.330002,235.570007,233.330002,235.059998,235.059998,45986200,Neutral,Positive
1738,2024-11-27,234.470001,235.690002,233.809998,234.929993,234.929993,33498400,Neutral,Positive


#### Saving all the datasets into csv files.

In [ ]:
from google.colab import files

# Save the dataframe to a CSV file
tsla_merged_data.to_csv('tsla_data_with_sentiment_analysis_from_18_to_24.csv', index=False)

# Download the file
files.download('tsla_data_with_sentiment_analysis_from_18_to_24.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Save the dataframe to a CSV file
aapl_merged_data.to_csv('aapl_data_with_sentiment_analysis_from_18_to_24.csv', index=False)

# Download the file
files.download('aapl_data_with_sentiment_analysis_from_18_to_24.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>